In [1]:
from mpi4py import MPI
from petsc4py import PETSc

from tqdm.notebook import tqdm_notebook

import numpy as np

import ufl

from dolfinx import fem, io, mesh, plot, la
from dolfinx.fem.petsc import assemble_matrix, create_vector, apply_lifting
from basix.ufl import element, mixed_element
from dolfinx.io import XDMFFile

import pyvista

https://www.wias-berlin.de/people/john/BETREUUNG/master_ding.pdf
https://www.youtube.com/watch?v=LflB3-bEK2M
http://redbkit.github.io/redbKIT/math/AdvDiffReact/#id

In [2]:
compounds = [ 'pt', 'th', 'at', 'fg', 'rp', 'ap', 'bp', 'fi' ] 
compounds_inital = [1400, 0, 2410, 7000, 1000, 50, 0, 0]
compounds_diffusion = {'pt': 5.21e-7, 'th' : 6.47e-7, 'at' : 5.57e-7, 'fg' : 3.10e-7, 'rp' : 2.5e-7, 'ap' : 2.5e-7, 'bp' : 0, 'fi' : 2.47e-7}
C = {'pt': 0, 'th' : 1, 'at' : 2, 'fg' : 3, 'rp' : 4, 'ap' : 5, 'bp' : 6, 'fi' : 7}
k_rp_th = 6.5e-10
k_ap_th = 3.69e-9
k_bp_th = 6.5e-10
k_at_th = 7.083e-3
k_th_fi = 59
k_th_Mfi = 3160
k_th_pa = 0.5
k_ap_pa = 0.3
k_bp = 1e4
k_bp_pa = 0.30
C_fi_50 = 60
C_th_50 = 0.911


In [3]:
# create mesh for calculations to take place in
msh = mesh.create_unit_square(MPI.COMM_WORLD, 16,16)

# define elements to be used for the velocity and the distribution
z_p1 = element("P", msh.basix_cell(), 1)
v_p2 = element("P", msh.basix_cell(), 2, shape=(msh.geometry.dim,))

# define mixed element and function space to be used when there are multiple compounds.
ME = mixed_element([z_p1, z_p1, z_p1, z_p1, z_p1, z_p1, z_p1, z_p1])
ZMixed = fem.FunctionSpace(msh, ME)

# Define function spaces to be used for the two variables
# Z = fem.FunctionSpace(msh, z_p1)
V = fem.FunctionSpace(msh, v_p2)

In [4]:
# define functions to be used for boundaries and initials
def initial_condition(x, a=5):
    return np.exp(-a * (x[0]**2 + x[1]**2))

def initial_condition_2(x, a=5):
    return np.exp(-a * (x[0]**2 + (x[1]-1)**2))

def flow(x):
    return np.stack((np.ones(x.shape[1]), np.zeros(x.shape[1]))) 
    
# Function to mark the lid (y = 1)
def lid(x):
    return np.isclose(x[1], 1)

def inflow_point(x):
    return np.isclose(x[0], 0) & np.isclose(x[1], 0.5)

# define time step variables 
T = 1.3          # final time
num_steps = 1300   # number of time steps
dt = T / num_steps # time step size

In [5]:
# # define boundary conditions
# inflow_dofs = fem.locate_dofs_geometrical(Z, lid)
# bc_inflow = fem.dirichletbc(PETSc.ScalarType(1), inflow_dofs, Z)
# bc_outlfow = fem.dirichletbc(PETSc.ScalarType(-1), inflow_dofs, Z)

# # order here is important as want to apply outflow before the inflow
# bcs = [bc_outlfow, bc_inflow]
bcs = []

In [6]:
# Create variables for results at current and previous timesteps
s_ = fem.Function(ZMixed)
# s_.interpolate(initial_condition)
s_n = fem.Function(ZMixed)
# s_n.interpolate(initial_condition)

sx_ = [s_.sub(x) for x in range(8)]
sx_n = [s_n.sub(x) for x in range(8)]

# s1_.name = "s1"
# s2_.name = "s2"
# s3_.name = "s3"

# sx_[0].interpolate(initial_condition)
# sx_n[0].interpolate(initial_condition)
# sx_[1].interpolate(initial_condition_2)
# sx_n[1].interpolate(initial_condition_2)

# s3_.x.array[:] = 0
# s3_n.x.array[:] = 0

for compound, x in C.items():
    val = compounds_inital[x]
    sx_[x].interpolate(lambda x: np.full((x.shape[1],), val))
    sx_n[x].interpolate(lambda x: np.full((x.shape[1],), val))
    s_.sub(x).name = compound




# very basic flow variable
u_n = fem.Function(V)
u_n.interpolate(flow)

# create trial and test functions
sx = ufl.TrialFunctions(ZMixed)
zx = ufl.TestFunctions(ZMixed)

In [7]:
# https://github.com/unifem/fenics-notes/blob/master/notebooks/advection-diffusion-reaction.ipynb
# https://jsdokken.com/dolfinx-tutorial/chapter2/diffusion_code.html

# other constants:
k_rp_th = fem.Constant(msh, PETSc.ScalarType(k_rp_th))
k_ap_th = fem.Constant(msh, PETSc.ScalarType(k_ap_th))
k_bp_th = fem.Constant(msh, PETSc.ScalarType(k_bp_th))
k_at_th = fem.Constant(msh, PETSc.ScalarType(k_at_th))
k_th_fi = fem.Constant(msh, PETSc.ScalarType(k_th_fi))
k_th_Mfi = fem.Constant(msh, PETSc.ScalarType(k_th_Mfi))
k_th_pa = fem.Constant(msh, PETSc.ScalarType(k_th_pa))
k_ap_pa = fem.Constant(msh, PETSc.ScalarType(k_ap_pa))
k_bp = fem.Constant(msh, PETSc.ScalarType(k_bp))
k_bp_pa = fem.Constant(msh, PETSc.ScalarType(k_bp_pa))
C_fi_50 = fem.Constant(msh, PETSc.ScalarType(C_fi_50))
C_th_50 = fem.Constant(msh, PETSc.ScalarType(C_th_50))

for key in compounds_diffusion.keys():
    compounds_diffusion[key] = fem.Constant(msh, PETSc.ScalarType(compounds_diffusion[key]))

# constants for calculations
rho = fem.Constant(msh, PETSc.ScalarType(1)) # advection coefficent
eps = fem.Constant(msh, PETSc.ScalarType(1)) # diffusion coefficent
k = fem.Constant(msh, dt)
fs = fem.Constant(msh, PETSc.ScalarType(0))
K = fem.Constant(msh, PETSc.ScalarType(1)) # reaction coefficent

F = 0

# https://fenicsproject.discourse.group/t/navier-stokes-and-convection-diffusion/7170
for compound, i in C.items():
    F += ( ( sx[i] - sx_n[i] ) / k ) * zx[i] * ufl.dx # advective terms
    F += ufl.dot( u_n, ufl.grad( sx_n[i] ) ) * zx[i] * ufl.dx # advective terms
    F += compounds_diffusion[compound] * ufl.dot( ufl.grad(sx[i] ), ufl.grad( zx[i] ) ) * ufl.dx # diffusive term
    F -= fs * zx[i] * ufl.dx

# reactive terms
F -= - k_rp_th * sx_[C['rp']] * sx_[C['pt']] * zx[C['pt']] * ufl.dx \
     - k_ap_th * sx_[C['ap']] * sx_[C['pt']] * zx[C['pt']] * ufl.dx \
     - k_bp_th * sx_[C['bp']] * sx_[C['pt']] * zx[C['pt']] * ufl.dx

F -= k_rp_th * sx_[C['rp']] * sx_[C['pt']] * zx[C['th']] * ufl.dx \
    + k_ap_th * sx_[C['ap']] * sx_[C['pt']] * zx[C['th']] * ufl.dx \
    + k_bp_th * sx_[C['bp']] * sx_[C['pt']] * zx[C['th']] * ufl.dx \
    - k_at_th * sx_[C['at']] * sx_[C['th']] * zx[C['th']] * ufl.dx

F -= - k_at_th * sx_[C['at']] * sx_[C['th']] * zx[C['at']] * ufl.dx

F -= - k_th_fi * sx_[C['th']] * ( sx_[C['fg']] / ( k_th_Mfi + sx_[C['fg']] ) ) * zx[C['fg']] * ufl.dx

F -= - k_th_pa * (sx_[C['th']] / (sx_[C['th']]**4 + C_th_50**4) ) * sx_[C['rp']] * zx[C['rp']] * ufl.dx \
    - k_ap_pa * sx_[C['rp']] * sx_[C['ap']] * zx[C['rp']] * ufl.dx \
    - k_bp_pa * sx_[C['rp']] * sx_[C['bp']] * zx[C['rp']] * ufl.dx

F -= k_th_pa * (sx_[C['th']]**4 / (sx_[C['th']]**4 + C_th_50**4) ) * sx_[C['rp']] * zx[C['ap']] * ufl.dx \
    + k_ap_pa * sx_[C['rp']] * sx_[C['ap']] * zx[C['ap']] * ufl.dx \
    + k_bp_pa * sx_[C['rp']] * sx_[C['bp']] * zx[C['ap']] * ufl.dx \
    - k_bp * (sx_[C['fi']]**4 / (sx_[C['fi']]**4 + C_fi_50**4) ) * sx_[C['ap']] * zx[C['ap']] * ufl.dx

F -= k_bp * (sx_[C['fi']]**4 / (sx_[C['fi']]**4 + C_fi_50**4) ) * sx_[C['ap']] * zx[C['bp']] * ufl.dx

F -= k_th_fi * sx_[C['th']] * ( sx_[C['fg']] / ( k_th_Mfi + sx_[C['fg']] ) ) * zx[C['fi']] * ufl.dx



a = fem.form(ufl.lhs(F))
L = fem.form(ufl.rhs(F))

# Just diffusion equation
# https://jsdokken.com/dolfinx-tutorial/chapte400r2/diffusion_code.html
# a = fem.form(s * z * ufl.dx + dt * ufl.dot(ufl.grad(s), ufl.grad(z)) * ufl.dx)
# L = fem.form((s_n + dt * fs) * z * ufl.dx)

# create parts of the equation that will not change between timesteps
# https://fenicsproject.discourse.group/t/how-to-set-bcs-for-petsc-matrices-and-petsc-vectors-in-dolfinx/4135
A = assemble_matrix(a, bcs=bcs)
A.assemble()
b = create_vector(L)

# create solver for the linear (matrix) problem
## Using petsc4py to create a linear solver
# As we have already assembled `a` into tbhe matrix `A`, we can no longer use the `dolfinx.fem.petsc.LinearProblem` class to solve the problem. Therefore, we create a linear algebra solver using PETSc, and assign the matrix `A` to the solver, and choose the solution strategy.
solver2 = PETSc.KSP().create(msh.comm)
solver2.setOperators(A)
solver2.setType(PETSc.KSP.Type.PREONLY)
solver2.getPC().setType(PETSc.PC.Type.LU)

In [8]:
# https://jsdokken.com/dolfinx-tutorial/chapter2/diffusion_code.html
# create output file
files = {}

for compound in C.keys():
    files[compound] = io.XDMFFile(msh.comm, f"results/sim/{compound}.xdmf", "w")
    files[compound].write_mesh(msh)
# xdmf0 = io.XDMFFile(msh.comm, "results/sim/0.xdmf", "w")
# xdmf1 = io.XDMFFile(msh.comm, "results/sim/1.xdmf", "w")
# xdmf2 = io.XDMFFile(msh.comm, "results/sim/2.xdmf", "w")

# xdmf0.write_mesh(msh)
# xdmf1.write_mesh(msh)
# xdmf2.write_mesh(msh)

# for compound, index in C.items():
#     files[compound].write_function(s_.sub(index),0)

In [9]:
# inital time
t = 0
for compound, index in C.items():
    val = s_.sub( index ).collapse()
    val.name = compound
    files[compound].write_function( val, t )

    print(f"[{compound}]->{s_.sub( index ).collapse().x.array[0]}")
# xdmf0.write_function(s_.sub(0), t)
# xdmf1.write_function(s_.sub(1), t)
# xdmf2.write_function(s_.sub(2), t)

for i in tqdm_notebook(range(num_steps)):
    t += dt # update time

    # https://jsdokken.com/dolfinx-tutorial/chapter2/diffusion_code.html
    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    fem.petsc.assemble_vector(b, L)
    apply_lifting(b, [a], [bcs])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    fem.set_bc(b, bcs)

    # solve
    solver2.solve(b, s_.vector)
    s_.x.scatter_forward() # correct for inacuracies 

    s_n.x.array[:] = s_.x.array # update previous step

    # xdmf0.write_function(s_.sub(0),t)
    # xdmf1.write_function(s_.sub(1), t)
    # xdmf2.write_function(s_.sub(2), t)
    outputting = True
    for compound, index in C.items():
        val = s_.sub( index ).collapse()
        val.name = compound
        files[compound].write_function( val, t )

        if (outputting == True):
            val = s_.sub( index ).collapse().x.array[0]
            if ( np.isnan(val) ):
                outputting = False
            print(f"[{compound}]->{s_.sub( index ).collapse().x.array[0]}")
    print("---------------------------------------------------")
# xdmf0.close()
# xdmf1.close()
# xdmf2.close()
for compound in C.keys():
    files[compound].close()

[pt]->1400.0
[th]->0.0
[at]->2410.0
[fg]->7000.0
[rp]->1000.0
[ap]->50.0
[bp]->0.0
[fi]->0.0


  0%|          | 0/1300 [00:00<?, ?it/s]

[pt]->1399.9999988317
[th]->1.1682999999999993e-06
[at]->2409.999999999998
[fg]->6999.999999999996
[rp]->984.9999999999991
[ap]->64.99999999999997
[bp]->0.0
[fi]->0.0
---------------------------------------------------
[pt]->1399.9999975995593
[th]->2.380497082922775e-06
[at]->2409.9999999800543
[fg]->6999.999999952506
[rp]->965.7924991646132
[ap]->84.20749999999987
[bp]->0.0
[fi]->4.749093503937e-08
---------------------------------------------------
[pt]->1399.9999962856714
[th]->3.6537490432897124e-06
[at]->2409.9999999394186
[fg]->6999.999999855735
[rp]->941.3944059336236
[ap]->108.60559156202098
[bp]->3.3051308208882274e-34
[fi]->1.442572042564006e-07
---------------------------------------------------
[pt]->1399.9999948679451
[th]->5.009104829157493e-06
[at]->2409.999999877047
[fg]->6999.999999707212
[rp]->910.7221945318107
[ap]->139.2778004669005
[bp]->3.662137046978018e-32
[fi]->2.9278066438129996e-07
---------------------------------------------------
[pt]->1399.999993319678
[

In [20]:
# visualisation
pyvista.set_jupyter_backend('client')

pyvista.start_xvfb()

dist1, test = ZMixed.sub(2).collapse()

topologyZ, cell_typesZ, geometryZ = plot.vtk_mesh(dist1)
grid0 = pyvista.UnstructuredGrid(topologyZ, cell_typesZ, geometryZ)
grid0.point_data["s0"] = s_n.sub(2).collapse().x.array

grid1 = pyvista.UnstructuredGrid(topologyZ, cell_typesZ, geometryZ)
grid1.point_data["s1"] = s_.sub(2).collapse().x.array

topologyV, cell_typesV, geometryV = plot.vtk_mesh(V)
values = np.zeros((geometryV.shape[0], 3), dtype=np.float64)
values[:, :len(u_n)] = u_n.x.array.real.reshape((geometryV.shape[0], len(u_n)))
velocity_grid = pyvista.UnstructuredGrid(topologyV, cell_typesV, geometryV)
velocity_grid["u"] = values
# Create a point cloud of glyphs
glyphs = velocity_grid.glyph(orient="u", factor=0.2)

plotter = pyvista.Plotter(notebook=True, shape=(1, 2))
plotter.subplot(0,0)
plotter.add_mesh(grid0, show_edges=True)
plotter.view_xy()
plotter.subplot (0,1)
plotter.add_mesh(grid1, show_edges=True)
plotter.view_xy()
#plotter.add_mesh(glyphs)

plotter.show()

Widget(value='<iframe src="http://localhost:43353/index.html?ui=P_0x7f50e1c10a60_1&reconnect=auto" class="pyvi…

In [48]:
# f = fem.Constant(msh, PETSc.ScalarType(0))
# a = u * v * ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
# L = (u_n + dt * f) * v * ufl.dx